In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("../데이터자료/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_Mid_irs','10Y_Mid_irs','1Y_Mid_crs','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [1]:
df["1Y_Mid_irs대비"] =df["1Y_Mid_irs"]-df["1Y_Mid_irs"].shift(1)
df["10Y_Mid_irs대비"] =df["10Y_Mid_irs"]-df["10Y_Mid_irs"].shift(1)
df["1Y_Mid_crs대비"] =df["1Y_Mid_crs"]-df["1Y_Mid_crs"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()

NameError: name 'df' is not defined

In [ ]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns


Index(['종가_ex', '1Y_Mid_irs', '10Y_Mid_irs', '1Y_Mid_crs', '국고1년', '통안364일',
       'Mid_ndf', 'M1_스왑포인트', '1Y_Mid_irs대비', '10Y_Mid_irs대비', '1Y_Mid_crs대비',
       '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트대비', '전날 종가_ex'],
      dtype='object')

In [ ]:
# 종속변수 독립변수 설정
df_Target = df["종가_ex"]
df_Data = df[['1Y_Mid_irs대비', '10Y_Mid_irs대비','1Y_Mid_crs대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트','전날 종가_ex']]
df_Data.feature= df_Data.columns
df_Data.feature


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_36668\3284639614.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_Mid_irs대비', '10Y_Mid_irs대비', '1Y_Mid_crs대비', '국고1년대비', '통안364일대비',
       'Mid_ndf대비', 'M1_스왑포인트', '전날 종가_ex'],
      dtype='object')

In [5]:
df_Data.boxplot(column='1Y_Mid_crs대비')

NameError: name 'df_Data' is not defined

In [8]:
# 스케일링 코드
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(df_Data)
df_Data_scaled= scaler.transform(df_Data)
df_scaled = pd.DataFrame (data = df_Data_scaled, columns=df_Data.feature)
df_scaled.index = df_Target.index
df_Data_scaled

array([[0.48743719, 0.4744898 , 0.41975309, ..., 0.50091111, 1.        ,
        0.37153652],
       [0.48743719, 0.48979592, 0.41975309, ..., 0.50312731, 0.98571429,
        0.38790932],
       [0.51256281, 0.50510204, 0.42592593, ..., 0.48983009, 0.98571429,
        0.39767003],
       ...,
       [0.49748744, 0.46938776, 0.43209877, ..., 0.4986949 , 0.56428571,
        0.94175063],
       [0.51758794, 0.53571429, 0.45061728, ..., 0.49475499, 0.55      ,
        0.95969773],
       [0.46733668, 0.43877551, 0.35802469, ..., 0.48909136, 0.55714286,
        0.90554156]])

In [18]:
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_scaled, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 6.318e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        20:51:32   Log-Likelihood:                -7661.9
No. Observations:                2459   AIC:                         1.533e+04
Df Residuals:                    2454   BIC:                         1.536e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1014.7394      3.128    324.433      0.000    1008.606    1020.873
1Y_Mid_crs대비   -50.2383      3.651    -13.759      0.000     -57.398     -43.079
국고1년대비           6.0281      1.639      3.678      0.000       2.814       9.242
Mid_ndf대비       26.5643      5.266      5.045      0.000      16.239      36.890
전날 종가_ex       316.7256      0.632    501.199      0.000     315.486     317.965
==============================================================================
Omnibus:                       45.348   Durbin-Watson:                   2.123
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.101
Skew:                           0.009   Prob(JB):                     1.65e-20
Kurtosis:                       3.943   Cond. No.                         71.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# p값높은 컬럼들 제외
df_Target = df["종가_ex"]
df_scaled = df_scaled[['1Y_Mid_crs대비', '국고1년대비', 'Mid_ndf대비','전날 종가_ex']]
feature_add_std = sm.add_constant(df_scaled, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 6.318e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        20:21:26   Log-Likelihood:                -7661.9
No. Observations:                2459   AIC:                         1.533e+04
Df Residuals:                    2454   BIC:                         1.536e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1014.7394      3.128    324.433      0.000    1008.606    1020.873
1Y_Mid_crs대비   -50.2383      3.651    -13.759      0.000     -57.398     -43.079
국고1년대비           6.0281      1.639      3.678      0.000       2.814       9.242
Mid_ndf대비       26.5643      5.266      5.045      0.000      16.239      36.890
전날 종가_ex       316.7256      0.632    501.199      0.000     315.486     317.965
==============================================================================
Omnibus:                       45.348   Durbin-Watson:                   2.123
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.101
Skew:                           0.009   Prob(JB):                     1.65e-20
Kurtosis:                       3.943   Cond. No.                         71.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_scaled,df_Target,test_size=0.2, random_state=16 )
lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse,mse

(5.121878719967354, 26.23364162205442)

In [17]:
from sklearn.model_selection import cross_val_score
neg_mse_scores = cross_val_score(lr, df_scaled, df_Target,scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 *neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_rmse,rmse_scores, neg_mse_scores

(8.131060132846212,
 array([ 4.0964659 ,  6.74277563,  5.72018172, 18.98432984,  5.11154757]),
 array([ -16.78103284,  -45.46502325,  -32.72047892, -360.40477941,
         -26.1279186 ]))